In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.cluster import KMeans
import seaborn as sns

/Users/albert/Programming/dano-hackatoon-2025/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
earnings = pd.read_csv('data/coins-come.csv', 
                       parse_dates=['Дата'],
                       sep=';',
                       encoding='utf-8-sig')
purchases = pd.read_csv('data/coins-out.csv', 
                        parse_dates=['Дата оформления заказа'], 
                        sep=';',
                        encoding='utf-8-sig')

/var/folders/qf/xy929rcx1xnc19y51wt8zct00000gn/T/ipykernel_3826/207400473.py:5: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  purchases = pd.read_csv('data/coins-out.csv',


In [3]:

# 1. Прогнозирование активности заработка бобров
# Агрегация по дням
daily_earnings = earnings.groupby(pd.Grouper(key='Дата', freq='D'))['Сумма вознаграждения'].sum().reset_index()

# Подготовка данных для Prophet
earnings_prophet = daily_earnings.rename(columns={'Дата': 'ds', 'Сумма вознаграждения': 'y'})


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [ ]:

# Создание и обучение модели
model = Prophet(weekly_seasonality=True, daily_seasonality=False)
model.fit(earnings_prophet)

# Прогноз на 30 дней
future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)

# Визуализация прогноза
fig1 = model.plot(forecast)
plt.title('Прогноз активности заработка бобров')
plt.xlabel('Дата')
plt.ylabel('Сумма вознаграждения')

# 2. Прогнозирование трат в Боброшопе
# Агрегация покупок по неделям
weekly_spending = purchases.resample('W-Mon', on='Дата оформления заказа')['Стоимость в валюте'].sum().reset_index()

# Прогноз с использованием ARIMA
from statsmodels.tsa.arima.model import ARIMA

# Обучение модели
model_arima = ARIMA(weekly_spending['Стоимость в валюте'], order=(1,1,1))
model_fit = model_arima.fit()

# Прогноз на 8 недель
forecast_spending = model_fit.forecast(steps=8)

# Визуализация
plt.figure()
plt.plot(weekly_spending['Дата оформления заказа'], weekly_spending['Стоимость в валюте'], label='История')
plt.plot(pd.date_range(start=weekly_spending['Дата оформления заказа'].iloc[-1], periods=8, freq='W'), 
         forecast_spending, label='Прогноз', linestyle='--')
plt.title('Прогноз трат в Боброшопе')
plt.legend()

# 3. Анализ взаимосвязи прогнозов
# Создание единого датафрейма с прогнозами
combined_forecast = pd.DataFrame({
    'date': forecast[['ds']].squeeze(),
    'earnings_forecast': forecast[['yhat']].squeeze(),
    'spending_forecast': forecast_spending.repeat(7)[:len(forecast)]  # Приведение к дневной гранулярности
})

# Расчет корреляции
correlation = combined_forecast[['earnings_forecast', 'spending_forecast']].corr()
print(f"\nКорреляция между прогнозами:\n{correlation}")

# Кластеризация временных паттернов
kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(combined_forecast[['earnings_forecast', 'spending_forecast']])
combined_forecast['cluster'] = clusters

# Визуализация кластеров
plt.figure()
sns.scatterplot(
    data=combined_forecast,
    x='earnings_forecast',
    y='spending_forecast',
    hue='cluster',
    palette='viridis',
    s=100
)
plt.title('Кластеризация паттернов активности')

# 4. Анализ сезонности
# Извлечение компонентов Prophet
fig2 = model.plot_components(forecast)

# Анализ недельной сезонности
earnings_prophet['day_of_week'] = earnings_prophet['ds'].dt.day_name()
weekly_pattern = earnings_prophet.groupby('day_of_week')['y'].mean()

plt.figure()
weekly_pattern.plot(kind='bar')
plt.title('Средний заработок по дням недели')

# 5. Оптимизация ассортимента на основе прогноза
# Расчет ожидаемого спроса
future_demand = pd.merge(
    purchases.groupby(['Наименование товара', pd.Grouper(key='Дата оформления заказа', freq='M')])
    .size().reset_index(name='count'),
    forecast[['ds']].rename(columns={'ds': 'Дата оформления заказа'}),
    how='right'
)

# Прогноз популярности товаров
from sklearn.ensemble import RandomForestRegressor

# Подготовка фичей
future_demand['month'] = future_demand['Дата оформления заказа'].dt.month
future_demand['year'] = future_demand['Дата оформления заказа'].dt.year

# Обучение модели
model_rf = RandomForestRegressor()
model_rf.fit(future_demand[['month', 'year']], future_demand['count'])

# Предсказание
future_demand['predicted_demand'] = model_rf.predict(future_demand[['month', 'year']])

# Рекомендации по топ-5 товарам
top_items = future_demand.groupby('Наименование товара')['predicted_demand'].sum().nlargest(5)
print("\nРекомендуемые к увеличению запаса товары:")
print(top_items)

plt.tight_layout()
plt.show()